In [50]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install pdpipe 
!pip install symspellpy
!pip install pycontractions

     |████████████████████████████████| 40 kB 1.5 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.5-py3-none-any.whl size=29946 sha256=016547466cf28ccfb2efb0ae3a20016295dc221e722b2eea9e58be95629850dd
  Stored in directory: /root/.cache/pip/wheels/90/00/4a/e3ae98003e155850dee523617ced7794e6e3da07d27e3802a0
  Created wheel for py-params: filename=py_params-0.9.6-py3-none-any.whl size=7088 sha256=69f613759fbf983026bc316fa5e339d8dd14a6924e6be95ecfbf6826de96f0d8
  Stored in directory: /root/.cache/pip/wheels/d1/4d/5a/7fbbdfb87bf3da12265e0d79dc19ce143652a09e3696189eb8
  Created wheel for params-flow: filename=params_flow-0.7.4-py3-none-any.whl size=16194 sha256=c1b967cb258bbbf9fedaa048dd5f3181c2e59a0b00f3d471235b6f8686205094
  Stored in directory: /root/.cache/pip/wheels/f4/90/ca/a5d500c088762773489e532acd0268226cf2597f211928f38a
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 47 kB 1.4 MB/s eta 0:00:011
  Created w

  Created wheel for language-check: filename=language_check-1.1-py3-none-any.whl size=90190895 sha256=0850eb1007396c09dd39487fd2d9ac41305e9470ea5cf811b4b6e7ed4f5cbeaf
  Stored in directory: /root/.cache/pip/wheels/ce/fe/32/3b83a67c4f1182f7f6aa134c1d04cdcd893072bdadb4f5a64c
Successfully built language-check


In [103]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



########################################################
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Model


try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert


#data pipeline 
import pdpipe as pdp

#Other Preprocessing
import re
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import string
# !pip install symspellpy
import pkg_resources
from symspellpy.symspellpy import SymSpell
from symspellpy import SymSpell, Verbosity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd

import nltk
nltk.download('punkt')

#Contraction Import
from pycontractions import Contractions

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl
/kaggle/input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [53]:
from bs4 import BeautifulSoup
import requests, json, csv
import re

def remove_wrong_abb(key,value):
    if(re.search('it means',value)):
        return False;
    elif(re.search('\*',value)):
        return False
    elif(value==""):
        return False
    elif(len(key)>7):
        return False
    else :
        return True
    
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            if(remove_wrong_abb(key,value)):
                if(re.search('\ -or- ',value)):
                    pos=re.search('\ -or- ',value).start()
                    slangdict[key]=value[:pos]
                else:
                    slangdict[key]=value

                    
# with open('myslang.json', 'w') as f:
#     json.dump(slangdict, f, indent=2)

w = csv.writer(open("myslang.csv", "w"))
for key, val in slangdict.items():
    w.writerow([key.lower(), val.lower()])
    
reader = csv.reader(open('myslang.csv', 'r'))
abbreviations = {}
for row in reader:
    k, v = row
    abbreviations[k] = v

In [54]:
#Version Edit: Sonam D.
sym_spell_4space = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell_4space.load_dictionary(dictionary_path, term_index=0, count_index=1)


sym_spell_misspelled = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell_misspelled.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell_misspelled.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

In [55]:
cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

[==================================================] 100.0% 387.1/387.1MB downloaded


In [56]:
#Version Edit: Sonam D.
def to_lower(text):
    text = text.lower()
    return text

#Version Edit: Anit G.
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r' ',text)

#Version Edit: Anit G.
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text)

#Version Edit: Anit G.
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

#Version Edit: Sonam D.
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

#Version Edit: Sonam D.
def remove_numbers(text):
    text = re.sub(r'\d+', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_mentions(text):
    text = re.sub(r'@\w*', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_punctuations(text):
    text = re.sub(r'([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_square_bracket(text):
    text = re.sub('\[.*?\]', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_angular_bracket(text):
    text = re.sub('<.*?>+', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_newline(text):
    text = re.sub('\n', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_words_with_numbers(text):
    text = re.sub('\w*\d\w*', ' ', text)
    return text
    
#Version Edit: Sonam D.
def hashtag_to_words(text):
    text = re.sub(r'##', '#', text)
    hash_pattern = re.compile(r"#\w*")
    hashtag_list = re.findall(hash_pattern,text)
    for hashtag in hashtag_list:
        hashtag = re.sub(r'#', '', hashtag)
        text = re.sub(hashtag, sym_spell_4space.word_segmentation(hashtag).corrected_string, text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'# ', '', text)
    return text

#Version Edit: Susanth D.
def remove_stopwords(text):
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        if token not in stopwords.words('english'):
            textop = textop + token + ' '
    return textop

#Version Edit: Sonam D.
def correct_misspelled_with_context(text):
    suggestions = sym_spell_misspelled.lookup_compound(text, max_edit_distance=2)
    text = str(suggestions[0])
    text = re.sub(r', \d', ' ', text)
#     text = remove_numbers(text)
    return text

#Version Edit: Sonam D.
def stemming_text(text):
    stemmer= PorterStemmer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + stemmer.stem(token) + ' '
    return textop

#Version Edit: Sonam D.
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + lemmatizer.lemmatize(token) + ' '
    return textop

#Version Edit: Anit G.
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

#Version Edit: Anit G.
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

#Version Edit: Sonam D.
# def handle_contractions(text):
#     text = re.sub(r"’", "'", text)
#     for word in text.split():
#         if word.lower() in contractions:
#             text = text.replace(word, contractions[word.lower()])
#     return(text)

#Version Edit: Saurabh M.
def removeRepeated(tweet):
    prev = ''
    tweet_new = ''
    for c in tweet:
        caps = False
        if c.isdigit():
            tweet_new += c
            continue
        if c.isalpha() == True:
            if ord(c) >= 65 and ord(c)<=90:
                caps = True
            c = c.lower()
            if c == prev:
                count += 1
            else:
                count = 1
                prev = c
            if count >= 3:
                continue
            if caps == True:
                tweet_new += c.upper()
            else:
                tweet_new += c
        else:
            tweet_new += c
    return tweet_new


def Expand_Contractions(text):
    return list(cont.expand_texts([text]))[0]

In [57]:
pipeline=pdp.ApplyByCols('text',to_lower,'text')
pipeline+=pdp.ApplyByCols('text',remove_newline,'text')
pipeline+=pdp.ApplyByCols('text',remove_URL,'text')
pipeline+=pdp.ApplyByCols('text',remove_html,'text')
pipeline+=pdp.ApplyByCols('text',remove_emoji,'text')
#pipeline+=pdp.ApplyByCols('text',hashtag_to_words,'text')
pipeline+=pdp.ApplyByCols('text',remove_words_with_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_mentions,'text')
pipeline+=pdp.ApplyByCols('text',remove_square_bracket,'text')
pipeline+=pdp.ApplyByCols('text',remove_angular_bracket,'text')
pipeline+=pdp.ApplyByCols('text',Expand_Contractions,'text')
pipeline+=pdp.ApplyByCols('text',remove_punctuations,'text')
pipeline+=pdp.ApplyByCols('text',removeRepeated,'text')
pipeline+=pdp.ApplyByCols('text',convert_abbrev_in_text,'text')
pipeline+=pdp.ApplyByCols('text',correct_misspelled_with_context,'text')
pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
pipeline+=pdp.ApplyByCols('text',remove_stopwords,'text')
#pipeline+=pdp.ApplyByCols('text',stemming_text,'text')
pipeline+=pdp.ApplyByCols('text',lemmatization,'text')




# df=pd.concat([train,test],sort=False)
train=pipeline(train)
test=pipeline(test)
test.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different city stay safe ever...
2,3,NaN,NaN,forest financial independence retire early spo...
3,9,NaN,NaN,apocalypse lighting spokane wildfire
4,11,NaN,NaN,typhoon decor kill china taiwan


In [98]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'

In [92]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [93]:
TEXT_COLUMN = 'text'
TARGET_COLUMN = 'target'
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'


In [100]:
x_train = train[TEXT_COLUMN].astype(str)
y_train = train[TARGET_COLUMN].values
x_test = test[TEXT_COLUMN].astype(str)

In [101]:
tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

In [104]:
crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

n unknown words (crawl):  173
n unknown words (glove):  137


In [105]:
embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

(11796, 600)

In [106]:
MAX_LEN = 300
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

In [107]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [108]:
def build_model(embedding_matrix):
    words = Input(shape=(None,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [109]:
NUM_MODELS = 2
BATCH_SIZE = 512
EPOCHS = 5

In [23]:
data_clean['location']=data_combined['location']
data_clean['keyword']=data_combined['keyword']
data_clean.head()

,text,text_tokenized,location,keyword
0,our deeds are the reason of this earthquake ma...,"[deeds, reason, earthquake, may, allah, forgiv...",NaN,NaN
1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]",NaN,NaN
2,all residents asked to shelter in place are be...,"[residents, asked, shelter, place, notified, o...",NaN,NaN
3,people receive wildfires evacuation orders in...,"[receive, wildfires, evacuation, orders, calif...",NaN,NaN
4,just got sent this photo from ruby alaska as s...,"[got, sent, photo, ruby, alaska, smoke, wildfi...",NaN,NaN


In [110]:
checkpoint_predictions = []
weights = []

for model_idx in range(NUM_MODELS):
    model = build_model(embedding_matrix)
    for global_epoch in range(EPOCHS):
        model.fit(
            x_train,y_train,
            batch_size=BATCH_SIZE,
            epochs=1,
            verbose=2
        )
        checkpoint_predictions.append(model.predict(x_test, batch_size=BATCH_SIZE))
        weights.append(2 ** global_epoch)

Train on 7613 samples
7613/7613 - 221s - loss: 0.5396
Train on 7613 samples
7613/7613 - 213s - loss: 0.4415
Train on 7613 samples
7613/7613 - 215s - loss: 0.4071
Train on 7613 samples
7613/7613 - 226s - loss: 0.3901
Train on 7613 samples
7613/7613 - 211s - loss: 0.3697
Train on 7613 samples
7613/7613 - 212s - loss: 0.5522
Train on 7613 samples
7613/7613 - 206s - loss: 0.4430
Train on 7613 samples
7613/7613 - 207s - loss: 0.4128
Train on 7613 samples
7613/7613 - 207s - loss: 0.3881
Train on 7613 samples
7613/7613 - 205s - loss: 0.3724


In [111]:
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

In [112]:
predictions.shape

(3263, 1)

In [115]:
sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
sub.iloc[:, 1] = (predictions > 0.5).astype(int)

In [116]:
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [117]:
sub.to_csv('submission.csv', index=False)

In [118]:
from collections import Counter
Counter(sub['target'])

Counter({1: 1199, 0: 2064})